In [3]:
# (test) Deploy locally on notebook intance 
# notebook instance - g5.12xl 
# kernel - conda_pytorch_p310

In [ ]:
import torch
import logging
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, AutoConfig
import os
##

code_tarname = 'acc_longchat13b_model'

In [ ]:
!pip install -r requirements.txt

In [ ]:
!git clone https://github.com/lm-sys/FastChat.git {code_tarname}

In [ ]:
!cd {code_tarname} && python -m pip install .

In [5]:
model = None
tokenizer = None


In [6]:
model_name = 'lmsys/longchat-13b-16k'

revision = 'main'
config = AutoConfig.from_pretrained(model_name, revision=revision)
from fastchat.model.llama_condense_monkey_patch import (
        replace_llama_with_condense,
    )

replace_llama_with_condense(config.rope_condense_ratio)

tokenizer = AutoTokenizer.from_pretrained(
        model_name, use_fast=True, revision=revision, trust_remote_code=True
    )
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True,
    max_memory={i: "20GiB" for i in range(4)}
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [9]:
data = 'what nutritional contents is in a tuna sandwich'

In [11]:

input_tokens = tokenizer(data, padding=True,
                         return_tensors="pt",
                         return_token_type_ids=False, # https://github.com/huggingface/peft/issues/506
                        ).to(torch.cuda.current_device())

output_ids = model.generate(
    **input_tokens, # **parameters
    do_sample=True,
    temperature=0.7,
    repetition_penalty=1.1,
    max_new_tokens=100,
)

output_ids = output_ids[0][len(input_tokens["input_ids"][0]) :]
generated_text = tokenizer.decode(
    output_ids, skip_special_tokens=True, spaces_between_special_tokens=False
)

# generated_text = tokenizer.batch_decode(output_tokens, ...)

In [12]:
generated_text

'. The first step in making a tuna sandwich is to prepare the ingredients, which includes slicing the bread, opening a can of tuna and draining the liquid, chopping vegetables such as lettuce, tomatoes, and onions, and measuring any additional ingredients like mayonnaise or mustard. Next, you would assemble the sandwich by spreading mayonnaise or mustard on one slice of bread, placing the tuna slices'